In [ ]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd())) # Include ../SSD in path

# Introduction to SSD

This code is much more complex than your prior assignment, and we recommend you to spend some time getting familiar with the code structure.
The "complex" code structure is made to simplify aspects of deep learning experimentation, and help you structure the usual "sphagetti" deep learning code.


All scripts in this code requires a configuration file. To **start training**, you can type:
```
python train.py configs/ssd300.py
```

## Configuration files
The key difference from previous starter codes is the use of configuration files. This enables us to change small parts of the experiment without changing hard-coded values (e.g. learning rate in previous assignments).

If you take a look in [`configs/ssd300.py`](../configs/ssd300.py) you will notice a large set of objects describing model architecture (`backbone` and `model`), the optimizer, dataset, data loading, and hyperparameters.

To load the config we can write the following:

In [ ]:
from ssd.utils import load_config
cfg = load_config("../configs/ssd300.py")

`cfg` supports access syntax, where all objects in `configs/ssd300.py` are accessible via their attribute name.



In [ ]:
print(cfg.model)

If we print `cfg.model`, notice that it returns a dictionary and not the model object itself (which is `SSD300` from [`ssd/modeling/ssd.py`](../ssd/modeling/ssd.py)). This is because the model is defined "lazily" (wrapped with a `LazyCall`).

To create the model, we have to instantiate it:

In [ ]:
from tops.config import instantiate
model = instantiate(cfg.model)
print(model)

Another example, we can load the first batch of the dataset and run a forward pass with the model:

In [ ]:
dataloader_train = instantiate(cfg.data_train.dataloader)
print(type(dataloader_train))
batch = next(iter(dataloader_train))
for key, item in batch.items():
    print(key, "has the shape:", item.shape)
gpu_transform = instantiate(cfg.data_train.gpu_transform)
batch = gpu_transform(batch)
bbox_delta, confidences = model(batch["image"])
print(f"The model predicted anchors with  bbox delta: {bbox_delta.shape} and confidences: {confidences.shape}")

You might ask yourself, why? At first sight, this seems very complicated rather than plain-old  hard coded values.

The reason is easy manipulation of experiments. If I want to run the same experiment, but with a different batch size, I can change it with the following:

In [ ]:
# Lets print the batch size of the original data loader:
print("Original batch size:", dataloader_train.batch_size)
cfg.train.batch_size = 2 # Setting the batch size to 2
dataloader_train = instantiate(cfg.data_train.dataloader)
print("New batch size:", dataloader_train.batch_size)

Another reason is **configuration inheritance**. E.g. for the last task, you are going to train VGG on the VOC dataset. 
This requires us to change the backbone and dataset, however, keep all other parameters the same (e.g. general configs (cfg.train), anchors, schedulers, optimizers, etc.).

Take a look in [`configs/voc_vgg.py`](../configs/voc_vgg.py) and notice that we inherit from the original config:
```
from .ssd300 import train, anchors, optimizer, schedulers, model, data_train, data_val
```
The only changes done are to the backbone, dataset and dataset transforms.

In [ ]:
cfg = load_config("../configs/voc_vgg.py")
model = instantiate(cfg.model)
print(model)

# Useful commands:


#### Training and evaluation
To start training:
```
python train.py  configs/ssd300.py
```

To starting training VGG on VOC:
```
python train.py  configs/voc_vgg.py
```

To only run evaluation:
```
python train.py  configs/ssd300.py --evaluate-only
```

#### Demo.py
For VOC:
```
python demo.py configs/voc_vgg.py demo/voc demo/voc_output
```

For MNIST:
```
python demo.py configs/ssd300.py demo/mnist demo/mnist_output
```


#### Runtime analysis:
```
python3 runtime_analysis.py configs/ssd300.py
```